In [1]:
import torch
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, pipeline

/home/kyle/miniconda3/envs/kne/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tokenizer = LlamaTokenizer.from_pretrained("TheBloke/vicuna-13B-1.1-HF")
# model = AutoModelForCausalLM.from_pretrained("TheBloke/vicuna-13B-1.1-HF", device_map="auto", torch_dtype=torch.float16).eval()

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-65b-hf")
model = AutoModelForCausalLM.from_pretrained("decapoda-research/llama-65b-hf", device_map="auto", torch_dtype=torch.float16).eval()

# tokenizer = AutoTokenizer.from_pretrained("mosaicml/mpt-7b-instruct")
# model = AutoModelForCausalLM.from_pretrained("mosaicml/mpt-7b-instruct", device_map="auto", torch_dtype=torch.float16, trust_remote_code=True).eval()

# tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")
# model = AutoModelForCausalLM.from_pretrained("facebook/opt-6.7b", device_map="auto", torch_dtype=torch.float16, trust_remote_code=True).eval()

# tokenizer = AutoTokenizer.from_pretrained("OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5")
# model = AutoModelForCausalLM.from_pretrained("OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5", device_map="auto", torch_dtype=torch.float16).eval()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 81/81 [02:35<00:00,  1.92s/it]


In [3]:
ag_news = load_dataset("ag_news", split="test")
ag_news[0]

Found cached dataset ag_news (/home/kyle/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


{'text': "Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.",
 'label': 2}

In [4]:
# ag_news_frame = ag_news.to_pandas()
# labels = sorted(ag_news_frame["label"].unique().tolist())
# sample = pd.concat([ag_news_frame[ag_news_frame["label"] == label].sample(4) for label in labels])
# sample["text"].values.tolist()

In [5]:
# current_summary = f"{ag_news[0]['text']}"

# task_prompt = f"""The assistant is to parpahrase/rewrite a given news summary in the style of a social media tweet. Here are some examples.

# Summary: India must recognize international realities: PM Prime Minister Manmohan Singh has responded to the Left #39;s criticism of his congratulatory call to US President George W. Bush by saying India must recognise international realities.
# Tweet: 🌍 PM Singh urges recognition of international realities! 🇮🇳🌐 Responding to criticism from the Left, he defends his congratulatory call to President Bush, emphasizing the need for India to acknowledge global realities. 👥🗣️ #InternationalRelations #ManmohanSingh 📰

# Summary: Quincy Carter being released by the Cowboys NEW YORK -- Tim Henman #39;s quarterfinal victory at the US Open was a microcosm of his career - long and brilliant in spurts, with an expected disappointment on the horizon.
# Tweet: LOL, Cowboys finally kicked Quincy Carter to the curb! 🏈 And hey, Tim Henman at the US Open? Classic Henman: flashy moments followed by an inevitable letdown. 😂🎾 #SameOldStory

# Summary: WellPoint net income increases 28 percent THOUSAND OAKS, Calif. -- WellPoint Health Networks Inc. #39;s third-quarter net income rose 28 percent as the managed-care company saw membership growth in key markets and double-digit revenue growth.
# Tweet: 💼📈 WellPoint reports impressive Q3 growth! 📊💰 The managed-care company's net income surged by 28% driven by membership expansion in key markets and substantial revenue growth in double digits. 👥💼 #WellPoint #FinancialResults 📰

# Summary: Microsoft releases fix for SP2-adware clash Microsoft has released a critical update for Windows Service Pack 2, designed to resolve an installation problem with a piece of adware -- but it maintains that the update isn #39;ta patch.
# Tweet: Microsoft releases fix for SP2-adware conflict! 🔧🔒 A critical update for Windows Service Pack 2 is now available, addressing an installation issue related to adware. Microsoft clarifies that the update is not a patch, but rather a resolution for the specific problem. #Microsoft #WindowsSP2 #SoftwareUpdate 📰

# Now rewrite the following news summary in the style of a social media tweet.

# Summary: {current_summary}
# Tweet:"""

# input_prompt = f"User: {task_prompt}Assistant:".replace("</s>", " ").replace("<s>", " ")
# tokenized_prompt = tokenizer.encode(input_prompt, return_tensors="pt").to(model.device)
# with torch.no_grad():
#     outputs = model.generate(
#         tokenized_prompt,
#         max_new_tokens=500,
#         length_penalty=0,
#         early_stopping=True,
#         return_dict_in_generate=True,
#         output_scores=True,
#     )
    
# generation = tokenizer.decode(outputs["sequences"][0])
# print(generation)

# # decode the scores


In [7]:
records = []

for index in tqdm(range(500)):
    current_entry = ag_news[index]
    current_summary = current_entry['text']
    task_prompt = f"""The assistant is to parpahrase/rewrite a given news summary in the style of a social media tweet. Here are some examples.

Summary: India must recognize international realities: PM Prime Minister Manmohan Singh has responded to the Left #39;s criticism of his congratulatory call to US President George W. Bush by saying India must recognise international realities.
Tweet: 🌍 PM Singh urges recognition of international realities! 🇮🇳🌐 Responding to criticism from the Left, he defends his congratulatory call to President Bush, emphasizing the need for India to acknowledge global realities. 👥🗣️ #InternationalRelations #ManmohanSingh 📰

Summary: WellPoint net income increases 28 percent THOUSAND OAKS, Calif. -- WellPoint Health Networks Inc. #39;s third-quarter net income rose 28 percent as the managed-care company saw membership growth in key markets and double-digit revenue growth.
Tweet: WellPoint reports impressive Q3 growth! The managed-care company's net income surged by 28% driven by membership expansion in key markets and substantial revenue growth in double digits. 👥💼 #WellPoint #FinancialResults 📰

Summary: Microsoft releases fix for SP2-adware clash Microsoft has released a critical update for Windows Service Pack 2, designed to resolve an installation problem with a piece of adware -- but it maintains that the update isn #39;ta patch.
Tweet: 💻 Microsoft releases fix for SP2-adware conflict! 🔧🔒 A critical update for Windows Service Pack 2 is now available, addressing an installation issue related to adware. Microsoft clarifies that the update is not a patch, but rather a resolution for the specific problem. #Microsoft #WindowsSP2 #SoftwareUpdate 📰

Summary: Quincy Carter being released by the Cowboys NEW YORK -- Tim Henman #39;s quarterfinal victory at the US Open was a microcosm of his career - long and brilliant in spurts, with an expected disappointment on the horizon.
Tweet: LOL, Cowboys finally kicked Quincy Carter to the curb! 🏈 And hey, Tim Henman at the US Open? Classic Henman: flashy moments followed by an inevitable letdown. 😂🎾 #SameOldStory

Summary: {current_summary}
Tweet: """

    # input_prompt = f"User: {task_prompt}Assistant:".replace("</s>", " ").replace("<s>", " ")
    input_prompt = task_prompt
    tokenized_prompt = tokenizer.encode(input_prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            tokenized_prompt,
            max_new_tokens=50,
            length_penalty=0,
            do_sample=False,
            early_stopping=True,
            return_dict_in_generate=True,
            output_scores=True,
        )

    generation = tokenizer.decode(outputs["sequences"][0]).split("Tweet:")[-1].split("</s>")[0].strip()
    records.append({
        "index": index,
        "original_summary": current_summary,
        "generated_summary": generation,
        "label": current_entry["label"]
    })
    
    print(f"\nOriginal Summary: {current_summary}")
    print(f"Generated Summary: {generation}")
        
    
records_frame = pd.DataFrame(records)
records_frame.to_csv("/home/kyle/repos/Parameter-Free-LM-Editing/datasets/ag_news_twitter/shifted_test_set_small_vicunna.csv", index=False)
records_frame

  0%|          | 1/500 [00:23<3:14:29, 23.39s/it]


Original Summary: Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.
Generated Summary: ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍


  0%|          | 1/500 [00:24<3:25:58, 24.77s/it]


KeyboardInterrupt: 

### MPT-7b

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "tiiuae/falcon-40b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

In [ ]:

for index in tqdm(range(200)):
    current_summary = ag_news[index]['text']
    task_prompt = f"""The assistant is to parpahrase/rewrite a given news summary in the style of a social media tweet. Here are some examples.

Summary: India must recognize international realities: PM Prime Minister Manmohan Singh has responded to the Left #39;s criticism of his congratulatory call to US President George W. Bush by saying India must recognise international realities.
Tweet: 🌍 PM Singh urges recognition of international realities! 🇮🇳🌐 Responding to criticism from the Left, he defends his congratulatory call to President Bush, emphasizing the need for India to acknowledge global realities. 👥🗣️ #InternationalRelations #ManmohanSingh 📰

Summary: Quincy Carter being released by the Cowboys NEW YORK -- Tim Henman #39;s quarterfinal victory at the US Open was a microcosm of his career - long and brilliant in spurts, with an expected disappointment on the horizon.
Tweet: LOL, Cowboys finally kicked Quincy Carter to the curb! 🏈 And hey, Tim Henman at the US Open? Classic Henman: flashy moments followed by an inevitable letdown. 😂🎾 #SameOldStory

Summary: WellPoint net income increases 28 percent THOUSAND OAKS, Calif. -- WellPoint Health Networks Inc. #39;s third-quarter net income rose 28 percent as the managed-care company saw membership growth in key markets and double-digit revenue growth.
Tweet: 💼📈 WellPoint reports impressive Q3 growth! 📊💰 The managed-care company's net income surged by 28% driven by membership expansion in key markets and substantial revenue growth in double digits. 👥💼 #WellPoint #FinancialResults 📰

Summary: Microsoft releases fix for SP2-adware clash Microsoft has released a critical update for Windows Service Pack 2, designed to resolve an installation problem with a piece of adware -- but it maintains that the update isn #39;ta patch.
Tweet: Microsoft releases fix for SP2-adware conflict! 🔧🔒 A critical update for Windows Service Pack 2 is now available, addressing an installation issue related to adware. Microsoft clarifies that the update is not a patch, but rather a resolution for the specific problem. #Microsoft #WindowsSP2 #SoftwareUpdate 📰

Now rewrite the following news summary in the style of a social media tweet.

Summary: {current_summary}
Tweet:"""

    generation = pipeline(
        task_prompt,
        max_new_tokens=200,
        do_sample=False,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )[0]["generated_text"]

    tweet = generation.split("Tweet:")[-1].split("\n")[0].strip()
    print("\nSummary:", current_summary)
    print(f"Tweet: {tweet}")

In [ ]:

# for index in range(10):
#     current_summary = ag_news[index]['text']
#     task_prompt = f"""The assistant is to parpahrase/rewrite a given news summary in the style of a social media tweet. Here are some examples.

# Summary: India must recognize international realities: PM Prime Minister Manmohan Singh has responded to the Left #39;s criticism of his congratulatory call to US President George W. Bush by saying India must recognise international realities.
# Tweet: 🌍 PM Singh urges recognition of international realities! 🇮🇳🌐 Responding to criticism from the Left, he defends his congratulatory call to President Bush, emphasizing the need for India to acknowledge global realities. 👥🗣️ #InternationalRelations #ManmohanSingh 📰

# Summary: WellPoint net income increases 28 percent THOUSAND OAKS, Calif. -- WellPoint Health Networks Inc. #39;s third-quarter net income rose 28 percent as the managed-care company saw membership growth in key markets and double-digit revenue growth.
# Tweet: 💼📈 WellPoint reports impressive Q3 growth! 📊💰 The managed-care company's net income surged by 28% driven by membership expansion in key markets and substantial revenue growth in double digits. 👥💼 #WellPoint #FinancialResults 📰

# Summary: Microsoft releases fix for SP2-adware clash Microsoft has released a critical update for Windows Service Pack 2, designed to resolve an installation problem with a piece of adware -- but it maintains that the update isn #39;ta patch.
# Tweet: 💻 Microsoft releases fix for SP2-adware conflict! 🔧🔒 A critical update for Windows Service Pack 2 is now available, addressing an installation issue related to adware. Microsoft clarifies that the update is not a patch, but rather a resolution for the specific problem. #Microsoft #WindowsSP2 #SoftwareUpdate 📰

# Summary: Quincy Carter being released by the Cowboys NEW YORK -- Tim Henman #39;s quarterfinal victory at the US Open was a microcosm of his career - long and brilliant in spurts, with an expected disappointment on the horizon.
# Tweet: LOL, Cowboys finally kicked Quincy Carter to the curb! 🏈 And hey, Tim Henman at the US Open? Classic Henman: flashy moments followed by an inevitable letdown. 😂🎾 #SameOldStory

# Now rewrite the following news summary in the style of a social media tweet.

# Summary: {current_summary}
# Tweet: """

#     input_prompt = f"<|prompter|>{task_prompt}<|endoftext|><|assistant|>".replace("</s>", " ").replace("<s>", " ")
#     tokenized_prompt = tokenizer.encode(task_prompt, return_tensors="pt").to(model.device)
#     with torch.no_grad():
#         outputs = model.generate(
#             tokenized_prompt,
#             max_new_tokens=500,
#             length_penalty=0,
#             early_stopping=True,
#             return_dict_in_generate=True,
#             output_scores=True,
#         )

#     generation = tokenizer.decode(outputs["sequences"][0]).split("Tweet: ")[-1].split("</s>")[0].strip()
#     print(generation)